In [1]:
import sys
import time
import pandas as pd

In [ ]:
def create_config_string(master_db,worker_database,set_name,ip,port,graph_node):
    
    config_str = []
    for worker_database,set_name,ip,port,graph_node in zip(worker_database,set_name,ip,port,graph_node):
        # config_str.append(f"[{name}] {data_type} {'NOT NULL' if nul == 'N' else 'NULL'}")
        config_str.append(f"fabric.graph.{graph_node}.uri=neo4j://{ip}:{port}" )
        config_str.append(f"fabric.graph.{graph_node}.database={worker_database}")
        config_str.append(f"fabric.graph.{graph_node}.name={set_name}" )

    config_str = "\n".join(config_str)
    
    str = f"""fabric.database.name={master_db} 
{config_str}    
        
"""

    return str

In [ ]:
def get_config_command(grouped_df):
    for index, row in grouped_df.iterrows():
        config_command = create_config_string(index,\
                                        row["worker_database"],\
                                        row["set_name"],\
                                        row["ip"],\
                                        row["port"],\
                                        row["graph_node"])
        # print(config_command)
        return config_command

In [ ]:
def get_config(filename):
    config_data = pd.read_excel(filename)
    grouped_df = config_data.groupby("master_database").agg(list)
    cmd = get_config_command(grouped_df)
    print(cmd)
    with open("output.txt", "w") as f:
        f.write(cmd)

In [ ]:
get_config('/content/fabric_config.xlsx')

In [2]:
config_data = pd.read_excel('/content/docker_config.xlsx')

In [3]:
config_data

,container_uuid,set_container_name,set_volume,set_http_port,set_bolt_port,set_password,set_plugins,set_dataset_folder
0,neo4j-db1,neo4j-db1,neo4j-db1-data,7477.0,7690.0,s3cr3t,"[""n10s""]",C:\Repo\TEN-XR\RnDgraph\data-science-graph\res...
1,neo4j-db2,neo4j-db2,neo4j-db2-data,7440.0,7688.0,password,"[""n10s""]",C:\Repo\TEN-XR\RnDgraph\data-science-graph\res...


In [4]:
grouped_df = config_data.groupby("container_uuid").agg(list)
grouped_df

,set_container_name,set_volume,set_http_port,set_bolt_port,set_password,set_plugins,set_dataset_folder
container_uuid,,,,,,,
neo4j-db1,[neo4j-db1],[neo4j-db1-data],[7477.0],[7690.0],[s3cr3t],"[[""n10s""]]",[C:\Repo\TEN-XR\RnDgraph\data-science-graph\re...
neo4j-db2,[neo4j-db2],[neo4j-db2-data],[7440.0],[7688.0],[password],"[[""n10s""]]",[C:\Repo\TEN-XR\RnDgraph\data-science-graph\re...


In [5]:
grouped_df.columns

Index(['set_container_name', 'set_volume', 'set_http_port', 'set_bolt_port',
       'set_password', 'set_plugins', 'set_dataset_folder'],
      dtype='object')

In [6]:
list(grouped_df.columns)

['set_container_name',
 'set_volume',
 'set_http_port',
 'set_bolt_port',
 'set_password',
 'set_plugins',
 'set_dataset_folder']

In [70]:
def create_config_string(container_uuid,set_container_name,set_volume,set_http_port,set_bolt_port,set_password,set_plugins,set_dataset_folder):
    
    config_str = []
    for set_container_name,set_volume,set_http_port,set_bolt_port,set_password,set_plugins,set_dataset_folder in zip(set_container_name,set_volume,set_http_port,set_bolt_port,set_password,set_plugins,set_dataset_folder):
        # print('create_config_string',set_container_name)
        # config_str.append(f"[{name}] {data_type} {'NOT NULL' if nul == 'N' else 'NULL'}")
        config_str.append(f"docker volume create {set_volume}")
        config_str.append(f"docker run  --name {set_container_name} --rm  --publish={set_http_port}:7474 --publish={set_bolt_port}:7687 -e NEO4J_AUTH=neo4j/{set_password} --volume={set_volume}:/data --env NEO4JLABS_PLUGINS='{set_plugins}' -v C:\neo4j\plugins:/plugins -v {set_dataset_folder}:/fabric_implementation  --env=NEO4J_ACCEPT_LICENSE_AGREEMENT=yes neo4j:4.3.0-enterprise")
        
    config_str = "\n".join(config_str)
    
    str = f"""
{config_str}
"""
  
    return str



def get_config_command(grouped_df):
    final_config = []
    for index, row in grouped_df.iterrows():
        # print(row)
        config_command = create_config_string(index,\
                                        row["set_container_name"],\
                                        row["set_volume"],\
                                        row["set_http_port"],\
                                        row["set_bolt_port"],\
                                        row["set_password"],\
                                        row["set_plugins"],\
                                        row["set_dataset_folder"])
        # print(config_command)
        final_config.append(config_command)
    return ' '.join(final_config)



In [71]:
print(get_config_command(grouped_df))


docker volume create neo4j-db1-data
docker run  --name neo4j-db1 --rm  --publish=7477.0:7474 --publish=7690.0:7687 -e NEO4J_AUTH=neo4j/s3cr3t --volume=neo4j-db1-data:/data --env NEO4JLABS_PLUGINS='["n10s"]' -v C:
eo4j\plugins:/plugins -v C:\Repo\TEN-XR\RnDgraph\data-science-graph\research\ontomanagement\fabric_implementation:/fabric_implementation  --env=NEO4J_ACCEPT_LICENSE_AGREEMENT=yes neo4j:4.3.0-enterprise
 
docker volume create neo4j-db2-data
docker run  --name neo4j-db2 --rm  --publish=7440.0:7474 --publish=7688.0:7687 -e NEO4J_AUTH=neo4j/password --volume=neo4j-db2-data:/data --env NEO4JLABS_PLUGINS='["n10s"]' -v C:
eo4j\plugins:/plugins -v C:\Repo\TEN-XR\RnDgraph\data-science-graph\research\ontomanagement\fabric_implementation:/fabric_implementation  --env=NEO4J_ACCEPT_LICENSE_AGREEMENT=yes neo4j:4.3.0-enterprise



In [9]:
for index, row in grouped_df.iterrows():
    print(row)

set_container_name                                          [neo4j-db1]
set_volume                                             [neo4j-db1-data]
set_http_port                                                  [7477.0]
set_bolt_port                                                  [7690.0]
set_password                                                   [s3cr3t]
set_plugins                                                  [["n10s"]]
set_dataset_folder    [C:\Repo\TEN-XR\RnDgraph\data-science-graph\re...
Name: neo4j-db1, dtype: object
set_container_name                                          [neo4j-db2]
set_volume                                             [neo4j-db2-data]
set_http_port                                                  [7440.0]
set_bolt_port                                                  [7688.0]
set_password                                                 [password]
set_plugins                                                  [["n10s"]]
set_dataset_folder    [C:\Repo\TE

In [ ]:
for index, row in grouped_df.iterrows():
    for worker_database,set_name,ip,port,graph_node in zip(row["worker_database"],row["set_name"],row["ip"],row["port"],row["graph_node"]):
        print(worker_database,set_name,ip,port,graph_node)

d3fend d3fend localhost 11006 0
odtmintegratedmodel odtmintegratedmodel localhost 11006 1
appsec appsec localhost 11006 2


In [ ]:
# def create_config_string():

#     pass

In [ ]:
fabric.graph.0.uri=neo4j://localhost:11006  
fabric.graph.0.database=d3fend  
fabric.graph.0.name=d3fend  
fabric.graph.1.uri=neo4j://localhost:11006  
fabric.graph.1.database=odtmintegratedmodel  
fabric.graph.1.name=odtmintegratedmodel  
fabric.graph.2.uri=neo4j://localhost:11006  
fabric.graph.2.database=appsec  
fabric.graph.2.name=appsec  

In [ ]:
fabric.graph.{graph_node}.uri=neo4j://{ip}:{port}
fabric.graph.{graph_node}.database={worker_database} 
fabric.graph.{graph_node}.name={set_name}

In [ ]:

def create_config_string(master_db,worker_database,set_name,ip,port,graph_node):
    
    config_str = []
    for worker_database,set_name,ip,port,graph_node in zip(worker_database,set_name,ip,port,graph_node):
        # config_str.append(f"[{name}] {data_type} {'NOT NULL' if nul == 'N' else 'NULL'}")
        config_str.append(f"fabric.graph.{graph_node}.uri=neo4j://{ip}:{port}" )
        config_str.append(f"fabric.graph.{graph_node}.database={worker_database}")
        config_str.append(f"fabric.graph.{graph_node}.name={set_name}" )

    config_str = "\n".join(config_str)
    
    str = f"""fabric.database.name={master_db} 
{config_str}    
        
"""

    return str

In [ ]:

def get_config_command(grouped_df):
    for index, row in grouped_df.iterrows():
        config_command = create_config_string(index,\
                                        row["worker_database"],\
                                        row["set_name"],\
                                        row["ip"],\
                                        row["port"],\
                                        row["graph_node"])
        # print(config_command)
        return config_command

fabric.database.name=fabric 
fabric.graph.0.uri=neo4j://localhost:11006
fabric.graph.0.database=d3fend
fabric.graph.0.name=d3fend
fabric.graph.1.uri=neo4j://localhost:11006
fabric.graph.1.database=odtmintegratedmodel
fabric.graph.1.name=odtmintegratedmodel
fabric.graph.2.uri=neo4j://localhost:11006
fabric.graph.2.database=appsec
fabric.graph.2.name=appsec    
        



In [ ]:
with open("fabric_config.txt", "w") as f:
    

In [ ]:
fabric.graph.0.uri=neo4j://localhost:11006  
fabric.graph.0.database=d3fend  
fabric.graph.0.name=d3fend  
fabric.graph.1.uri=neo4j://localhost:11006  
fabric.graph.1.database=odtmintegratedmodel  
fabric.graph.1.name=odtmintegratedmodel  
fabric.graph.2.uri=neo4j://localhost:11006  
fabric.graph.2.database=appsec  
fabric.graph.2.name=appsec  